<a href="https://colab.research.google.com/github/wav0lav/lspower-24-25/blob/main/Pearson%26Spearman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import random
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import dates as d
import datetime
from datetime import date, timedelta
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.lines as mlines
import re
import seaborn as sns
from sklearn.cluster import KMeans
!pip install kneed
from kneed import KneeLocator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import recall_score

Mount the Drive

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


# Cleaning and Aggregating Daily Data



# RUN THIS BEFORE EVERY PROGRAM

getting data sets

In [ ]:
random_state_val = 42

In [ ]:
daily_averages_df = pd.read_csv('/content/gdrive/Shareddrives/LSPowerData/aggregatedDailyAverages.csv')
individual_data_df = pd.read_csv('/content/gdrive/Shareddrives/LSPowerData/aggregatedIndividualData.csv', low_memory=False)
merged_df = pd.read_csv('/content/gdrive/Shareddrives/LSPowerData/aggregatedIndividualAndDailyUpdated.csv', low_memory=False)

In [ ]:

merged_df

,level_0,index,Timestamp (US/Eastern),CH4 (%),CO2 (%),O2 (%),Bal. Gas (%),LFG Flow (SCFM),LFG Temperature (F),Applied Vacuum - PA (in. H2O),Available Vacuum - PB (in. H2O),Static Pressure - PS (in. H2O),Valve Position (%),Liquid Column (feet),Well,day,Follow Up Priority
0,568,7580,2022-12-26 07:02:50,57.3,39.9,0.3,2.5,55.4,85.8,-6.3,-5.8,NaN,74.9,NaN,TCLC-412,2022-12-26,Yellow
1,569,7581,2022-12-28 11:18:09,57.8,41.0,0.2,1.1,69.5,89.9,-10.2,-14.0,NaN,74.9,NaN,TCLC-412,2022-12-28,Yellow
2,570,7582,2022-12-28 14:23:31,58.0,41.1,0.1,0.8,67.9,89.4,-9.7,-14.2,NaN,73.7,NaN,TCLC-412,2022-12-28,Yellow
3,571,7583,2022-12-28 17:32:20,58.0,41.2,0.0,0.8,69.4,88.8,-10.0,-14.3,NaN,74.5,NaN,TCLC-412,2022-12-28,Yellow
4,572,7584,2022-12-28 20:48:17,55.0,39.2,0.3,5.4,70.0,88.5,-10.5,-13.4,NaN,75.3,NaN,TCLC-412,2022-12-28,Yellow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744033,1186320,1193332,2024-08-31 20:49:19,52.1,43.4,0.0,4.6,NaN,75.9,-4.2,-14.2,NaN,19.1,12.7,TCGW-001,2024-08-31,Green
744034,1186321,1193333,2024-08-31 21:09:58,NaN,NaN,NaN,NaN,NaN,76.0,-4.6,-14.3,NaN,18.0,NaN,TCGW-001,2024-08-31,Green
744035,1186322,1193334,2024-08-31 22:06:49,NaN,NaN,NaN,NaN,NaN,76.7,-5.0,-14.3,NaN,18.0,NaN,TCGW-001,2024-08-31,Green
744036,1186323,1193335,2024-08-31 23:03:48,NaN,NaN,NaN,NaN,NaN,75.4,-5.0,-14.1,NaN,18.0,NaN,TCGW-001,2024-08-31,Green


**Drop any rows with NAN**

In [ ]:
merged_df

,level_0,index,Timestamp (US/Eastern),CH4 (%),CO2 (%),O2 (%),Bal. Gas (%),LFG Flow (SCFM),LFG Temperature (F),Applied Vacuum - PA (in. H2O),Available Vacuum - PB (in. H2O),Static Pressure - PS (in. H2O),Valve Position (%),Liquid Column (feet),Well,day,Follow Up Priority
0,568,7580,2022-12-26 07:02:50,57.3,39.9,0.3,2.5,55.4,85.8,-6.3,-5.8,NaN,74.9,NaN,TCLC-412,2022-12-26,Yellow
1,569,7581,2022-12-28 11:18:09,57.8,41.0,0.2,1.1,69.5,89.9,-10.2,-14.0,NaN,74.9,NaN,TCLC-412,2022-12-28,Yellow
2,570,7582,2022-12-28 14:23:31,58.0,41.1,0.1,0.8,67.9,89.4,-9.7,-14.2,NaN,73.7,NaN,TCLC-412,2022-12-28,Yellow
3,571,7583,2022-12-28 17:32:20,58.0,41.2,0.0,0.8,69.4,88.8,-10.0,-14.3,NaN,74.5,NaN,TCLC-412,2022-12-28,Yellow
4,572,7584,2022-12-28 20:48:17,55.0,39.2,0.3,5.4,70.0,88.5,-10.5,-13.4,NaN,75.3,NaN,TCLC-412,2022-12-28,Yellow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744033,1186320,1193332,2024-08-31 20:49:19,52.1,43.4,0.0,4.6,NaN,75.9,-4.2,-14.2,NaN,19.1,12.7,TCGW-001,2024-08-31,Green
744034,1186321,1193333,2024-08-31 21:09:58,NaN,NaN,NaN,NaN,NaN,76.0,-4.6,-14.3,NaN,18.0,NaN,TCGW-001,2024-08-31,Green
744035,1186322,1193334,2024-08-31 22:06:49,NaN,NaN,NaN,NaN,NaN,76.7,-5.0,-14.3,NaN,18.0,NaN,TCGW-001,2024-08-31,Green
744036,1186323,1193335,2024-08-31 23:03:48,NaN,NaN,NaN,NaN,NaN,75.4,-5.0,-14.1,NaN,18.0,NaN,TCGW-001,2024-08-31,Green


In [ ]:
merged_df_wo_nan = merged_df.copy()
# merged_df_temp = merged_df_temp[~merged_df_temp['CH4 (%)'] == np.nan]
merged_df_wo_nan


nan_indices_gas = merged_df[merged_df[['CH4 (%)', 'CO2 (%)', 'O2 (%)', 'Bal. Gas (%)']].isna().any(axis=1)].index
nan_indices_select = merged_df[merged_df[['CH4 (%)', 'CO2 (%)', 'O2 (%)', 'Bal. Gas (%)', 'LFG Flow (SCFM)', 'LFG Temperature (F)', 'Applied Vacuum - PA (in. H2O)', 'Available Vacuum - PB (in. H2O)', 'Liquid Column (feet)']].isna().any(axis=1)].index


# this is gets rid of any nan gas values
merged_gas =  merged_df.copy()
merged_gas = merged_gas.drop(nan_indices_gas)

# this is gets rid of any nan values for CH4, CO2, O2, Bal. Gas, PA, PB, Liquid Column
merged_select =  merged_df.copy()
merged_select = merged_select.drop(nan_indices_select)

Shuffling rows

In [ ]:
# Shuffling Rows for ML
merged_gas = merged_gas.sample(frac=1)
merged_select = merged_select.sample(frac=1)

In [ ]:
merged_gas

,level_0,index,Timestamp (US/Eastern),CH4 (%),CO2 (%),O2 (%),Bal. Gas (%),LFG Flow (SCFM),LFG Temperature (F),Applied Vacuum - PA (in. H2O),Available Vacuum - PB (in. H2O),Static Pressure - PS (in. H2O),Valve Position (%),Liquid Column (feet),Well,day,Follow Up Priority
185565,242859,249871,2023-01-05 17:36:00,57.4,40.5,0.2,1.9,66.1,85.3,-13.4,-20.1,NaN,74.5,NaN,TCLC-19,2023-01-05,Green
112738,3376,10388,2023-10-25 18:37:22,53.8,39.1,1.0,6.1,54.9,89.7,-6.4,-11.6,NaN,62.7,NaN,TCLC-412,2023-10-25,Green
357963,496429,503441,2024-04-03 23:11:54,56.7,41.5,0.1,1.8,NaN,53.1,-22.7,-20.5,NaN,44.3,<10,TCGW-061,2024-04-03,Green
684622,1092533,1099545,2024-04-24 18:09:22,51.2,43.3,0.8,4.6,4.4,81.2,-23.2,-24.3,NaN,10.6,<9,TCGW-008,2024-04-24,Green
167521,163119,170131,2023-05-19 10:48:45,56.7,40.3,0.3,2.7,NaN,72.5,-4.5,-10.7,NaN,16.1,NaN,TCLC-29,2023-05-19,Green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179511,214190,221202,2023-12-27 00:56:12,55.0,39.4,0.2,5.4,3.3,54.7,-1.1,-5.5,NaN,18.0,NaN,TCLC-24,2023-12-27,Green
556505,854069,861081,2024-01-05 12:59:18,54.8,44.5,0.0,0.7,NaN,74.0,-8.8,-6.8,NaN,100.0,11.8,TCGW-027,2024-01-05,Green
16216,233200,240212,2023-02-07 04:08:17,53.9,37.5,1.2,7.4,3.4,40.0,-3.6,-22.0,NaN,18.0,NaN,TCLC-22,2023-02-07,Yellow
425120,631329,638341,2024-07-14 19:12:08,53.0,45.0,0.0,1.9,24.5,137.6,-25.7,-24.5,NaN,97.6,48.7,TCGW-047,2024-07-14,Green


In [ ]:
merged_select

,level_0,index,Timestamp (US/Eastern),CH4 (%),CO2 (%),O2 (%),Bal. Gas (%),LFG Flow (SCFM),LFG Temperature (F),Applied Vacuum - PA (in. H2O),Available Vacuum - PB (in. H2O),Static Pressure - PS (in. H2O),Valve Position (%),Liquid Column (feet),Well,day,Follow Up Priority
615185,965071,972083,2024-01-14 08:33:12,53.9,44.7,0.2,1.2,0.0,58.8,-8.2,-6.2,NaN,100.0,<11,TCGW-018,2024-01-14,Green
506503,759194,766206,2024-07-09 01:05:19,53.7,45.2,0.0,1.1,18.7,132.6,-3.8,-25.5,NaN,24.7,<10.5,TCGW-037A,2024-07-09,Green
237627,328554,335566,2024-02-02 15:57:24,56.4,41.6,0.0,2.0,17.0,120.6,-5.5,-5.9,NaN,100.0,<13,TCGW-113,2024-02-02,Green
465884,695635,702647,2024-05-05 02:06:18,53.2,45.2,0.0,1.6,13.8,98.5,-10.9,-26.8,NaN,28.6,<12,TCGW-041,2024-05-05,Green
692620,1105287,1112299,2024-04-26 03:22:46,55.3,42.0,0.1,2.6,8.1,56.0,-3.1,-25.7,NaN,14.5,<15,TCGW-006A,2024-04-26,Green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272211,366935,373947,2024-03-22 15:01:20,53.2,43.9,0.1,2.8,18.9,96.7,-3.7,-9.0,NaN,37.3,<13,TCGW-108,2024-03-22,Green
432459,644083,651095,2024-07-11 10:47:55,53.1,43.1,0.2,3.7,6.1,99.5,-5.0,-24.2,NaN,22.4,<18,TCGW-045,2024-07-11,Green
355852,488789,495801,2024-08-07 12:27:45,55.3,41.3,0.0,3.4,5.6,100.4,-4.3,-26.9,NaN,23.5,21.6,TCGW-063,2024-08-07,Green
490721,742429,749441,2023-12-30 04:56:42,51.9,46.5,0.0,1.6,20.5,93.0,-6.1,-1.7,NaN,92.2,<10,TCGW-036,2023-12-30,Green


functions for processing

In [ ]:
# THIS BLOCK HAS ALL THE FUNCTIONS FOR DATA FILTERING FOR INDIVIDUAL WELLS

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

# filters individual well data given a filename for an individual well
def data_filtering_individual(filename):
  search_path = '/content/gdrive/Shareddrives/LSPowerData/IndividualWellData'
  # Search for the file
  file_path = os.path.join(search_path, filename)
  df = pd.read_csv(file_path, header=6)

  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan}, inplace=True)
  df.replace({'--': np.nan}, inplace=True) #fake value can ignore and add plot limits if needed

  df.loc[:, 'CH4 (%)'] = df['CH4 (%)'].astype(float)
  df.loc[:, 'O2 (%)'] = df['O2 (%)'].astype(float)
  df.loc[:, 'LFG Temperature (F)'] = df['LFG Temperature (F)'].astype(float)
  df.loc[:, 'Bal. Gas (%)'] = df['Bal. Gas (%)'].astype(float)
  df.loc[:, 'Applied Vacuum - PA (in. H2O)'] = df['Applied Vacuum - PA (in. H2O)'].astype(float)
  df['Datetime'] = pd.to_datetime(df['Timestamp (US/Eastern)'])
  return df

  # Get Indices

# given the overall aggregated data frame + an individual well, outputs all the times the well is yellow
def find_yellow_times(df, well):
  y =  ((df[df['Follow Up Priority'] == 'Yellow'].index))
  n =  ((df[df['Well'] == well].index))
  intersect = intersection(y, n)

  # time_list is a list of days where the well has yellow readings
  time_list = []
  for i in range(len(intersect)):
    time_list.append(df.loc[intersect[i], 'day'])
  return time_list

# Find indices in individual well data of specific start and end time
def get_time_period(df, start, end):
  # Getse start + end time (!!)
  dt = df['Datetime']
  time1 = pd.to_datetime(start) #date 1
  index = np.searchsorted(dt, time1)

  # TODO: make an end variable
  time2 = pd.to_datetime(end) # date 2
  index2 = np.searchsorted(dt, time2) + 1
  return (index, index2)

# Find indices in individual well data from when well was broken
# Useful for plotting
def broken_list(df, time_list):
  dt = df['Datetime']
  broken = []
  for i in range(len(time_list)):
    time = pd.to_datetime(time_list[i]) #date 1
    broken.append(np.searchsorted(dt, time))
  return broken


def find_intersection(lists):
    """Finds the intersection of multiple lists."""

    if not lists:
        return []

    # Convert lists to sets for efficient intersection
    sets = [set(lst) for lst in lists]

    # Find the intersection using the intersection method
    result = sets[0].intersection(*sets[1:])

    return list(result)


def find_union(lists):
    """Finds the intersection of multiple lists."""

    if not lists:
        return []

    # Convert lists to sets for efficient intersection
    sets = [set(lst) for lst in lists]

    # Find the intersection using the intersection method
    result = sets[0].union(*sets[1:])

    return list(result)


def matthews_correlation(tp, tn, fp, fn):
  tp, tn, fp, fn = float(tp), float(tn), float(fp), float(fn)
  num = tp * tn - fp * fn
  denom = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
  return num/denom


# Pearson Coefficient

In [ ]:
import os
from scipy.stats import pearsonr

def data_filtering_individual(file_path):
    """
    Reads and preprocesses a single CSV file.
    """
    df = pd.read_csv(file_path, header=6)

    # Replace placeholders with NaN
    df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)

    # Convert relevant columns to floats
    float_columns = ['CH4 (%)', 'O2 (%)', 'LFG Temperature (F)', 'Bal. Gas (%)',
                     'Applied Vacuum - PA (in. H2O)']
    for col in float_columns:
        if col in df.columns:
            df[col] = df[col].astype(float)

    # Add a datetime column
    if 'Timestamp (US/Eastern)' in df.columns:
        df['Datetime'] = pd.to_datetime(df['Timestamp (US/Eastern)'])

    return df

def calculate_correlation_matrices(df):
    """
    Computes Pearson and Spearman correlation matrices for the given DataFrame.
    """
    # Select numerical columns only
    numerical_df = df.select_dtypes(include=[float, int])

    # Calculate Pearson and Spearman correlation matrices
    pearson_corr = numerical_df.corr(method='pearson')
    spearman_corr = numerical_df.corr(method='spearman')

    return pearson_corr, spearman_corr

def process_all_files(folder_path):
    """
    Processes all CSV files in a folder and computes pairwise correlations.
    Returns the average Pearson and Spearman correlation matrices across all files.
    """
    pearson_sum = None
    spearman_sum = None
    file_count = 0

    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            try:
                # Read and filter data
                df = data_filtering_individual(file_path)

                # Calculate correlation matrices
                pearson_corr, spearman_corr = calculate_correlation_matrices(df)

                # Initialize sums if first file
                if pearson_sum is None:
                    pearson_sum = pearson_corr.copy()
                    spearman_sum = spearman_corr.copy()
                else:
                    pearson_sum = pearson_sum.add(pearson_corr, fill_value=0)
                    spearman_sum = spearman_sum.add(spearman_corr, fill_value=0)

                file_count += 1
                print(f"Processed {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    if file_count == 0:
        raise ValueError("No valid files were processed.")

    # Average the correlation matrices
    avg_pearson = pearson_sum / file_count
    avg_spearman = spearman_sum / file_count

    return avg_pearson, avg_spearman

# Example Usage
if __name__ == "__main__":
    folder_path = "/content/gdrive/Shareddrives/LSPowerData/IndividualWellData"
    avg_pearson, avg_spearman = process_all_files(folder_path)

    print("Average Pearson Correlation Matrix:")
    print(avg_pearson)
    print("\nAverage Spearman Correlation Matrix:")
    print(avg_spearman)


<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)


Processed HEADER-1-09-24-2022-11_47-09-24-2024-11_47.csv


<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)


Processed TCLC-412-09-24-2022-11_53-09-24-2024-11_53.csv


<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)
<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)
<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_

...

<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)


Processed HEADER-6-09-25-2022-12_48-09-25-2024-12_48.csv


<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)
<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)
<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_

Processed HEADER-5-09-25-2022-12_48-09-25-2024-12_48.csv
Processed HEADER-4-09-25-2022-12_48-09-25-2024-12_48.csv
Processed HEADER-3-09-25-2022-12_48-09-25-2024-12_48.csv
Processed HEADER-2-09-25-2022-12_48-09-25-2024-12_48.csv
Average Pearson Correlation Matrix:
                                 Applied Vacuum - PA (in. H2O)  \
Applied Vacuum - PA (in. H2O)                         0.929825   
Available Vacuum - PB (in. H2O)                       0.070022   
Bal. Gas (%)                                          0.008162   
CH4 (%)                                              -0.007260   
LFG Temperature (F)                                  -0.231124   
Liquid Column (feet)                                       NaN   
O2 (%)                                                0.026150   
Static Pressure - PS (in. H2O)                             NaN   
Valve Position (%)                                   -0.464222   

                                 Available Vacuum - PB (in. H2O)  \
Applied

<ipython-input-14-a1e2b0d070c4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'NR': np.nan, 'missing': np.nan, 'F': np.nan, '--': np.nan}, inplace=True)


Save code:

In [ ]:
# Save the correlation matrices as CSV files
avg_pearson.to_csv("average_pearson_correlation.csv", index=True)
avg_spearman.to_csv("average_spearman_correlation.csv", index=True)

print("Correlation matrices saved as CSV files:")
print("- average_pearson_correlation.csv")
print("- average_spearman_correlation.csv")


Correlation matrices saved as CSV files:
- average_pearson_correlation.csv
- average_spearman_correlation.csv
